In [ ]:
# Notebook 3: Hindi Healthcare RAG Experiments

# ============================================================================
# Cell 1 - Setup and Imports
# ============================================================================

import sys
import os
sys.path.append('..')

import json
import time
import logging
from pathlib import Path
import pandas as pd
from datetime import datetime
import openai
from dotenv import load_dotenv
from typing import Dict, List, Tuple, Any, Optional

from src.rag_system import MultilingualRAG
from langchain.schema import Document
import numpy as np

# SET THE API KEY IN ENVIRONMENT
OPEN_API_KEY = "<Your key>"
os.environ["OPENAI_API_KEY"] = OPEN_API_KEY  # Make it available to rag_system

if OPEN_API_KEY:
    openai.api_key = OPEN_API_KEY
    print(f"✓ OpenAI API Key set (version: {openai.__version__})")
else:
    print("⚠️  OpenAI API Key not found - will use mock responses")

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

ROOT_DIR = Path('..').resolve()
PROCESSED_DATA_DIR = ROOT_DIR / 'data' / 'processed'
EMBEDDINGS_DIR = ROOT_DIR / 'data' / 'embeddings'
RESULTS_DIR = ROOT_DIR / 'results'
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Root directory: {ROOT_DIR}")
print(f"Results directory: {RESULTS_DIR}")
print(f"Environment variable set: {bool(os.getenv('OPENAI_API_KEY'))}")

# ============================================================================
# Cell 2 - Load Processed Documents
# ============================================================================

# Load processed documents
processed_path = PROCESSED_DATA_DIR / 'processed_documents.json'

with open(processed_path, 'r', encoding='utf-8') as f:
    docs_data = json.load(f)

# Convert back to Document objects
documents = []
for doc_dict in docs_data:
    doc = Document(
        page_content=doc_dict['content'],
        metadata=doc_dict['metadata']
    )
    documents.append(doc)

print(f"✓ Loaded {len(documents)} documents")


# ============================================================================
# Cell 3 - Define Hindi Test Questions
# ============================================================================

# Comprehensive Hindi test questions organized by medical domain
HINDI_TEST_QUESTIONS = {
    'diabetes': [
        {
            'hindi': 'मधुमेह के प्रमुख लक्षण क्या हैं?',
            'english': 'What are the main symptoms of diabetes?',
            'category': 'symptoms',
            'complexity': 'simple'
        },
        {
            'hindi': 'टाइप 2 डायबिटीज को कैसे नियंत्रित करें?',
            'english': 'How to control type 2 diabetes?',
            'category': 'management',
            'complexity': 'moderate'
        },
        {
            'hindi': 'मधुमेह रोगियों के लिए कौन सी दवाएं उपलब्ध हैं?',
            'english': 'What medications are available for diabetes patients?',
            'category': 'medication',
            'complexity': 'moderate'
        },
        {
            'hindi': 'मधुमेह में खान-पान का क्या ध्यान रखना चाहिए?',
            'english': 'What dietary precautions should be taken in diabetes?',
            'category': 'lifestyle',
            'complexity': 'simple'
        },
    ],
    'heart_disease': [
        {
            'hindi': 'हृदय रोग से बचाव के उपाय क्या हैं?',
            'english': 'What are the preventive measures for heart disease?',
            'category': 'prevention',
            'complexity': 'simple'
        },
        {
            'hindi': 'दिल का दौरा पड़ने पर तुरंत क्या करना चाहिए?',
            'english': 'What should be done immediately during a heart attack?',
            'category': 'emergency',
            'complexity': 'complex'
        },
        {
            'hindi': 'कोलेस्ट्रॉल कम करने के प्राकृतिक तरीके क्या हैं?',
            'english': 'What are natural ways to reduce cholesterol?',
            'category': 'lifestyle',
            'complexity': 'moderate'
        },
    ],
    'hypertension': [
        {
            'hindi': 'उच्च रक्तचाप के लिए कौन सी दवाएं सबसे प्रभावी हैं?',
            'english': 'Which medications are most effective for high blood pressure?',
            'category': 'medication',
            'complexity': 'moderate'
        },
        {
            'hindi': 'हाई ब्लड प्रेशर में नमक का सेवन कितना होना चाहिए?',
            'english': 'How much salt should be consumed in high blood pressure?',
            'category': 'lifestyle',
            'complexity': 'simple'
        },
        {
            'hindi': 'रक्तचाप की दवाओं के दुष्प्रभाव क्या हैं?',
            'english': 'What are the side effects of blood pressure medications?',
            'category': 'medication',
            'complexity': 'complex'
        },
    ],
    'general': [
        {
            'hindi': 'वयस्कों के लिए कौन से टीके जरूरी हैं?',
            'english': 'Which vaccines are necessary for adults?',
            'category': 'vaccination',
            'complexity': 'simple'
        },
        {
            'hindi': 'स्वस्थ जीवनशैली के लिए क्या सुझाव हैं?',
            'english': 'What are recommendations for a healthy lifestyle?',
            'category': 'lifestyle',
            'complexity': 'simple'
        },
    ]
}

# Prepare questions for experiment
all_questions = []
for domain, questions in HINDI_TEST_QUESTIONS.items():
    all_questions.extend(questions)

hindi_questions = [q['hindi'] for q in all_questions]
english_references = [q['english'] for q in all_questions]

print(f"\n✓ Loaded {len(hindi_questions)} Hindi test questions")
print("\nSample questions:")
for i, (hi, en) in enumerate(zip(hindi_questions[:3], english_references[:3]), 1):
    print(f"\n{i}. {hi}")
    print(f"   ({en})")


# ============================================================================
# Cell 4 - Initialize RAG System
# ============================================================================

print("\nInitializing RAG system...")

rag_system = MultilingualRAG(
    embedding_model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
    vector_store_type='faiss',
    persist_directory=str(EMBEDDINGS_DIR / 'multilingual')
)

# Load existing vector store
vector_store_path = EMBEDDINGS_DIR / 'multilingual' / 'faiss'
if not vector_store_path.exists():
    print("Creating vector store...")
    rag_system.create_vector_store(documents)
else:
    print("Loading existing vector store...")
    rag_system.load_vector_store()

print("✓ RAG system ready")

# ============================================================================
# DIAGNOSTIC CELL - Check Embedding Consistency
# ============================================================================

print("🔍 EMBEDDING MODEL DIAGNOSTIC")
print("=" * 80)

# Check what's in the RAG system
print(f"\nRAG System Configuration:")
print(f"  Embedding model: {rag_system.embedding_model}")
print(f"  Vector store type: {rag_system.vector_store_type}")
print(f"  Use multilingual: {rag_system.use_multilingual_embeddings}")

# Check the actual embeddings object
print(f"\nEmbedding Object:")
print(f"  Type: {type(rag_system.embeddings)}")
if hasattr(rag_system.embeddings, 'model_name'):
    print(f"  Model name: {rag_system.embeddings.model_name}")

# Test: Embed the same text in both approaches
test_query_hindi = "मधुमेह के लक्षण"
test_query_english = "symptoms of diabetes"

print(f"\n🧪 TEST: Embedding the same concept")
print(f"  Hindi: {test_query_hindi}")
print(f"  English: {test_query_english}")

# Get embeddings
from src.utils import translate_text

hindi_embedding = rag_system.embeddings.embed_query(test_query_hindi)
english_embedding = rag_system.embeddings.embed_query(test_query_english)
translated_query = translate_text(test_query_hindi, src='hi', dest='en')
print(f"  Translated: {translated_query}")
translated_embedding = rag_system.embeddings.embed_query(translated_query)

print(f"\nEmbedding dimensions:")
print(f"  Hindi: {len(hindi_embedding)}")
print(f"  English: {len(english_embedding)}")
print(f"  Translated: {len(translated_embedding)}")

# Calculate similarity
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

sim_hindi_english = cosine_similarity(
    [hindi_embedding], [english_embedding]
)[0][0]

sim_hindi_translated = cosine_similarity(
    [hindi_embedding], [translated_embedding]
)[0][0]

sim_english_translated = cosine_similarity(
    [english_embedding], [translated_embedding]
)[0][0]

print(f"\n📊 Embedding Similarities:")
print(f"  Hindi ↔ English: {sim_hindi_english:.4f}")
print(f"  Hindi ↔ Translated: {sim_hindi_translated:.4f}")
print(f"  English ↔ Translated: {sim_english_translated:.4f}")

if sim_hindi_english < 0.7:
    print("\n⚠️  WARNING: Low similarity between Hindi and English!")
    print("   This suggests the multilingual model may not be working well")
    print("   for this concept.")

# Test actual retrieval
print(f"\n🔍 TEST: Actual Retrieval Comparison")
print("=" * 80)

# Multilingual retrieval (Hindi query directly)
rag_system.use_multilingual_embeddings = True
docs_multi = rag_system.vector_store.similarity_search(test_query_hindi, k=3)

print(f"\n🔵 Multilingual (Hindi query): '{test_query_hindi}'")
print(f"  Retrieved {len(docs_multi)} docs")
for i, doc in enumerate(docs_multi[:2], 1):
    print(f"  [{i}] {doc.page_content[:100]}...")
    print(f"      Source: {doc.metadata.get('source', 'Unknown')}")

# Translation-based retrieval (Translated query)
rag_system.use_multilingual_embeddings = False
docs_trans = rag_system.vector_store.similarity_search(translated_query, k=3)

print(f"\n🔴 Translation (English query): '{translated_query}'")
print(f"  Retrieved {len(docs_trans)} docs")
for i, doc in enumerate(docs_trans[:2], 1):
    print(f"  [{i}] {doc.page_content[:100]}...")
    print(f"      Source: {doc.metadata.get('source', 'Unknown')}")

# Check overlap
multi_contents = [d.page_content[:200] for d in docs_multi]
trans_contents = [d.page_content[:200] for d in docs_trans]
overlap = len(set(multi_contents) & set(trans_contents))

print(f"\n📊 Retrieval Overlap: {overlap}/{min(len(docs_multi), len(docs_trans))}")

if overlap == 0:
    print("⚠️  ZERO OVERLAP! Approaches are retrieving completely different content!")
    print("   This explains the quality differences.")

# ============================================================================
# DIAGNOSTIC CELL 2 - Check Vector Store Content
# ============================================================================

print("\n🔍 VECTOR STORE CONTENT CHECK")
print("=" * 80)

# Sample some documents from the vector store
import random

# Get total document count
if hasattr(rag_system.vector_store, 'index'):
    # FAISS
    total_docs = rag_system.vector_store.index.ntotal
    print(f"Total documents in FAISS: {total_docs}")
elif hasattr(rag_system.vector_store, '_collection'):
    # Chroma
    total_docs = rag_system.vector_store._collection.count()
    print(f"Total documents in Chroma: {total_docs}")

# Sample random documents
print(f"\nSampling 5 random documents:")
test_queries = ["diabetes", "heart disease", "blood pressure", "medication", "treatment"]

for query in test_queries[:3]:
    docs = rag_system.vector_store.similarity_search(query, k=1)
    if docs:
        print(f"\nQuery: '{query}'")
        print(f"  Content: {docs[0].page_content[:150]}...")
        print(f"  Source: {docs[0].metadata.get('source', 'Unknown')}")
        print(f"  File: {docs[0].metadata.get('file', 'Unknown')}")

# Check if Hindi queries work
print(f"\n\n🔍 Testing Hindi Query Retrieval:")
hindi_queries = ["मधुमेह", "हृदय रोग", "रक्तचाप"]

for hindi_q in hindi_queries:
    docs = rag_system.vector_store.similarity_search(hindi_q, k=1)
    if docs:
        print(f"\nHindi query: '{hindi_q}'")
        print(f"  Retrieved: {docs[0].page_content[:100]}...")
    else:
        print(f"\n⚠️  No results for Hindi query: '{hindi_q}'")

# ============================================================================
# Cell 5 - Enhanced Experiment Runner Class
# ============================================================================

class HindiHealthcareRAGExperiment:
    """
    Experiment runner comparing two approaches for Hindi healthcare QA:
    1. Multilingual embeddings: Hindi→Retrieve directly→Generate in Hindi
    2. Translation-based: Hindi→English→Retrieve→Generate→Hindi
    """
    
    def __init__(self, rag_system, has_openai_key: bool = False):
        self.rag_system = rag_system
        self.has_openai_key = has_openai_key
        self.system_prompt = (
            "You are a helpful healthcare information assistant. "
            "Based on the provided context from CDC, FDA, and medical literature, "
            "answer the question accurately and concisely in the SAME LANGUAGE as the question. "
            "If the context doesn't contain relevant information, say so clearly."
        )
    
    def _retrieve_with_metrics(
        self, 
        query: str,
        use_multilingual: bool,
        k: int = 5
    ) -> Tuple[List, str, float, List[float]]:
        """Retrieve documents with detailed metrics"""
        start = time.time()
        
        self.rag_system.use_multilingual_embeddings = use_multilingual
        
        # Retrieve with scores
        try:
            docs_with_scores = self.rag_system.vector_store.similarity_search_with_score(
                query, k=k
            )
        except (AttributeError, NotImplementedError):
            # Fallback if similarity_search_with_score not available
            docs = self.rag_system.vector_store.similarity_search(query, k=k)
            docs_with_scores = [(doc, 1.0 - i*0.05) for i, doc in enumerate(docs)]
        
        documents = [doc for doc, score in docs_with_scores]
        scores = [float(score) for doc, score in docs_with_scores]
        
        retrieval_time = time.time() - start
        
        return documents, query, retrieval_time, scores
    
    def _generate_with_metrics(
        self,
        question: str,
        documents: List,
        target_language: str = "hi"
    ) -> Tuple[str, float]:
        """Generate response in target language"""
        start = time.time()
        
        response, gen_time = self.rag_system.generate_response(
            question, documents, target_language, self.system_prompt
        )
        
        total_time = time.time() - start
        return response, total_time
    
    def _translate_for_eval(
        self, 
        text: str, 
        src_lang: str = "hi",
        dest_lang: str = "en"
    ) -> Tuple[str, float]:
        """Translate text for evaluation purposes"""
        from src.utils import translate_text
        start = time.time()
        translated = translate_text(text, src=src_lang, dest=dest_lang)
        trans_time = time.time() - start
        
        return translated, trans_time
    
    def run_approach_multilingual(
        self,
        hindi_question: str,
        k: int = 5
    ) -> Dict[str, Any]:
        """Approach A: Direct multilingual embeddings"""
        result = {}
        
        try:
            # Step 1: Retrieve using Hindi query with multilingual embeddings
            docs, eff_query, ret_time, scores = self._retrieve_with_metrics(
                query=hindi_question,
                use_multilingual=True,
                k=k
            )
            
            result['multilingual_retrieval_time'] = ret_time
            result['multilingual_retrieved_chunks'] = [d.page_content[:300] for d in docs]
            result['multilingual_chunk_scores'] = scores
            result['multilingual_chunk_sources'] = [d.metadata.get('source', 'Unknown') for d in docs]
            result['multilingual_effective_query'] = eff_query
            
            # Step 2: Generate response in Hindi
            response_hindi, gen_time = self._generate_with_metrics(
                hindi_question, docs, target_language="hi"
            )
            
            result['multilingual_response_hindi'] = response_hindi
            result['multilingual_generation_time'] = gen_time
            
            # Step 3: Translate response to English for evaluation
            response_en, trans_time = self._translate_for_eval(
                response_hindi, src_lang="hi", dest_lang="en"
            )
            
            result['multilingual_response_english_for_eval'] = response_en
            result['multilingual_response_translation_time'] = trans_time
            
            # Total time
            result['multilingual_total_time'] = ret_time + gen_time + trans_time
            
            logging.info(f"  ✓ Multilingual: {result['multilingual_total_time']:.2f}s")
            
        except Exception as e:
            logging.error(f"  ✗ Multilingual failed: {str(e)}")
            result = self._get_error_result('multilingual', str(e))
        
        return result
    
    def run_approach_translation(
        self,
        hindi_question: str,
        k: int = 5
    ) -> Dict[str, Any]:
        """Approach B: Translation-based pipeline"""
        result = {}
        
        try:
            # Step 1: Translate Hindi question to English
            question_en, q_trans_time = self._translate_for_eval(
                hindi_question, src_lang="hi", dest_lang="en"
            )
            
            result['translation_question_english'] = question_en
            result['translation_question_translation_time'] = q_trans_time
            
            # Step 2: Retrieve using English query
            docs, eff_query, ret_time, scores = self._retrieve_with_metrics(
                query=question_en,
                use_multilingual=False,
                k=k
            )
            
            result['translation_retrieval_time'] = ret_time
            result['translation_retrieved_chunks'] = [d.page_content[:300] for d in docs]
            result['translation_chunk_scores'] = scores
            result['translation_chunk_sources'] = [d.metadata.get('source', 'Unknown') for d in docs]
            result['translation_effective_query'] = eff_query
            
            # Step 3: Generate response in English
            response_en, gen_time = self._generate_with_metrics(
                question_en, docs, target_language="en"
            )
            
            result['translation_response_english'] = response_en
            result['translation_generation_time'] = gen_time
            
            # Step 4: Translate English response to Hindi
            response_hindi, resp_trans_time = self._translate_for_eval(
                response_en, src_lang="en", dest_lang="hi"
            )
            
            result['translation_response_hindi'] = response_hindi
            result['translation_response_translation_time'] = resp_trans_time
            
            # Store English response for evaluation
            result['translation_response_english_for_eval'] = response_en
            
            # Total time
            result['translation_total_time'] = (
                q_trans_time + ret_time + gen_time + resp_trans_time
            )
            
            logging.info(f"  ✓ Translation: {result['translation_total_time']:.2f}s")
            
        except Exception as e:
            logging.error(f"  ✗ Translation failed: {str(e)}")
            result = self._get_error_result('translation', str(e))
        
        return result
    
    def _get_error_result(self, approach: str, error_msg: str) -> Dict[str, Any]:
        """Generate error result dictionary"""
        return {
            f'{approach}_error': error_msg,
            f'{approach}_retrieval_time': 0.0,
            f'{approach}_generation_time': 0.0,
            f'{approach}_total_time': 0.0,
            f'{approach}_response_hindi': f"ERROR: {error_msg}",
            f'{approach}_response_english_for_eval': f"ERROR: {error_msg}",
            f'{approach}_retrieved_chunks': [],
            f'{approach}_chunk_scores': [],
            f'{approach}_chunk_sources': [],
        }
    
    def run_single_question(
        self,
        hindi_question: str,
        question_id: str,
        english_equivalent: Optional[str] = None,
        k: int = 5
    ) -> Dict[str, Any]:
        """Run complete experiment for a single Hindi question"""
        result = {
            'question_id': question_id,
            'question_hindi': hindi_question,
            'question_english_reference': english_equivalent or '',
            'system_prompt': self.system_prompt,
            'num_chunks_retrieved': k,
        }
        
        # Run Approach A: Multilingual embeddings
        print(f"  Running Approach A (Multilingual embeddings)...")
        result.update(self.run_approach_multilingual(hindi_question, k))
        
        # Run Approach B: Translation-based
        print(f"  Running Approach B (Translation pipeline)...")
        result.update(self.run_approach_translation(hindi_question, k))
        
        return result
    
    def run_experiments(
        self,
        hindi_questions: List[str],
        english_references: Optional[List[str]] = None,
        output_path: str = None,
        k: int = 5
    ) -> pd.DataFrame:
        """Run experiments for multiple Hindi questions"""
        if english_references and len(english_references) != len(hindi_questions):
            raise ValueError("Number of English references must match Hindi questions")
        
        results = []
        
        print(f"\nRunning Hindi Healthcare RAG Experiments")
        print(f"Total questions: {len(hindi_questions)}")
        print("=" * 80)
        
        for idx, hindi_q in enumerate(hindi_questions, 1):
            print(f"\n[{idx}/{len(hindi_questions)}] {hindi_q}")
            
            question_id = f"Q{idx:03d}"
            english_ref = english_references[idx-1] if english_references else None
            
            result = self.run_single_question(
                hindi_question=hindi_q,
                question_id=question_id,
                english_equivalent=english_ref,
                k=k
            )
            results.append(result)
            
            time.sleep(0.3)
        
        df = pd.DataFrame(results)
        
        # Save results
        if output_path:
            Path(output_path).parent.mkdir(parents=True, exist_ok=True)
            df.to_csv(output_path, index=False, encoding='utf-8')
            print("\n" + "=" * 80)
            print(f"✓ Saved results to {output_path}")
            print(f"  Shape: {df.shape}")
        
        return df


# ============================================================================
# Cell 6 - Run Experiments
# ============================================================================

# Initialize experiment runner
experiment_runner = HindiHealthcareRAGExperiment(
    rag_system=rag_system,
    has_openai_key=bool(OPEN_API_KEY)
)

# Run experiments
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_path = RESULTS_DIR / f'hindi_healthcare_rag_{timestamp}.csv'

df_results = experiment_runner.run_experiments(
    hindi_questions=hindi_questions,
    english_references=english_references,
    output_path=str(output_path),
    k=5
)

# Display summary
print("\n" + "=" * 80)
print("EXPERIMENT SUMMARY")
print("=" * 80)
print(f"\nTotal questions processed: {len(df_results)}")
print(f"\nAverage times:")
print(f"  Multilingual approach: {df_results['multilingual_total_time'].mean():.2f}s")
print(f"  Translation approach:  {df_results['translation_total_time'].mean():.2f}s")

print(f"\nTime breakdown (Multilingual):")
print(f"  Retrieval:  {df_results['multilingual_retrieval_time'].mean():.2f}s")
print(f"  Generation: {df_results['multilingual_generation_time'].mean():.2f}s")
print(f"  Translation to EN (for eval): {df_results['multilingual_response_translation_time'].mean():.2f}s")

print(f"\nTime breakdown (Translation):")
print(f"  Query translation (HI→EN): {df_results['translation_question_translation_time'].mean():.2f}s")
print(f"  Retrieval:  {df_results['translation_retrieval_time'].mean():.2f}s")
print(f"  Generation: {df_results['translation_generation_time'].mean():.2f}s")
print(f"  Response translation (EN→HI): {df_results['translation_response_translation_time'].mean():.2f}s")

print("\n" + "=" * 80)
print("✓ Experiments complete! Proceed to Notebook 4 for evaluation.")
print("=" * 80)

✓ OpenAI API Key set (version: 1.35.0)
Root directory: C:\Users\Boris\Desktop\code\multilingual-rag
Results directory: C:\Users\Boris\Desktop\code\multilingual-rag\results
Environment variable set: True
✓ Loaded 13487 documents

✓ Loaded 12 Hindi test questions

Sample questions:

1. मधुमेह के प्रमुख लक्षण क्या हैं?
   (What are the main symptoms of diabetes?)

2. टाइप 2 डायबिटीज को कैसे नियंत्रित करें?
   (How to control type 2 diabetes?)

3. मधुमेह रोगियों के लिए कौन सी दवाएं उपलब्ध हैं?
   (What medications are available for diabetes patients?)

Initializing RAG system...


2025-10-13 01:42:07,263 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
C:\Users\Boris\Desktop\code\multilingual-rag\mul-rag\Lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading existing vector store...


2025-10-13 01:42:10,461 - INFO - Loading faiss with AVX2 support.
2025-10-13 01:42:10,462 - INFO - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2025-10-13 01:42:10,462 - INFO - Loading faiss.
2025-10-13 01:42:10,479 - INFO - Successfully loaded faiss.
2025-10-13 01:42:10,764 - INFO - Loaded faiss vector store


✓ RAG system ready
🔍 EMBEDDING MODEL DIAGNOSTIC

RAG System Configuration:
  Embedding model: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
  Vector store type: faiss
  Use multilingual: True

Embedding Object:
  Type: <class 'langchain_community.embeddings.huggingface.HuggingFaceEmbeddings'>
  Model name: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

🧪 TEST: Embedding the same concept
  Hindi: मधुमेह के लक्षण
  English: symptoms of diabetes


...

2025-10-13 01:43:45,139 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-13 01:43:47,472 - INFO -   ✓ Multilingual: 4.35s


  Running Approach B (Translation pipeline)...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-10-13 01:43:48,821 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-13 01:43:49,807 - INFO -   ✓ Translation: 2.33s



✓ Saved results to C:\Users\Boris\Desktop\code\multilingual-rag\results\hindi_healthcare_rag_20251013_014211.csv
  Shape: (12, 28)

EXPERIMENT SUMMARY

Total questions processed: 12

Average times:
  Multilingual approach: 5.01s
  Translation approach:  2.87s

Time breakdown (Multilingual):
  Retrieval:  0.04s
  Generation: 3.86s
  Translation to EN (for eval): 1.11s

Time breakdown (Translation):
  Query translation (HI→EN): 0.55s
  Retrieval:  0.03s
  Generation: 1.16s
  Response translation (EN→HI): 1.13s

✓ Experiments complete! Proceed to Notebook 4 for evaluation.


In [2]:
# ============================================================================
# Cell 7 - Response Verification with Back-Translation (Hindi Only)
# ============================================================================

from src.utils import translate_text
import time
import numpy as np

print("\n" + "=" * 100)
print("RESPONSE VERIFICATION WITH BACK-TRANSLATION")
print("=" * 100)

def assess_response_quality(response: str) -> dict:
    """Simple quality checks"""
    quality = {
        'length_ok': len(response) > 100,
        'not_mock': 'mock' not in response.lower() and '[' not in response[:10],
        'has_medical_terms': any(term in response.lower() for term in [
            'मधुमेह', 'हृदय', 'रक्त', 'दवा', 'उपचार', 'स्वास्थ्य', 'रोग', 'लक्षण',
            'diabetes', 'heart', 'blood', 'medication', 'treatment'
        ]),
        'coherent': len(response.split()) > 10,
        'word_count': len(response.split())
    }
    quality['score'] = sum([
        quality['length_ok'], quality['not_mock'], 
        quality['has_medical_terms'], quality['coherent']
    ]) / 4.0
    return quality

# Process each question
for idx, row in df_results.iterrows():
    print(f"\n{'='*100}")
    print(f"QUESTION {idx+1}/{len(df_results)}")
    print(f"{'='*100}")
    print(f"\n📝 Hindi: {row['question_hindi']}")
    print(f"📝 English: {row['question_english_reference']}")
    
    # MULTILINGUAL APPROACH
    print(f"\n{'-'*100}")
    print("🔵 MULTILINGUAL EMBEDDINGS")
    print(f"{'-'*100}")
    
    multi_hi = row['multilingual_response_hindi']
    print(f"\n✅ Hindi Response:")
    print(multi_hi)
    
    print(f"\n🔄 Back-translating to English...")
    multi_back = translate_text(multi_hi, src='hi', dest='en')
    print(f"\n📖 Back-translated:")
    print(multi_back)
    
    quality_multi = assess_response_quality(multi_hi)
    print(f"\n📈 Quality: {quality_multi['score']:.0%} ({quality_multi['word_count']} words)")
    print(f"   ✓ Length adequate: {'✅' if quality_multi['length_ok'] else '❌'}")
    print(f"   ✓ Not mock: {'✅' if quality_multi['not_mock'] else '❌'}")
    print(f"   ✓ Has medical terms: {'✅' if quality_multi['has_medical_terms'] else '❌'}")
    print(f"   ✓ Coherent: {'✅' if quality_multi['coherent'] else '❌'}")
    
    # TRANSLATION APPROACH  
    print(f"\n{'-'*100}")
    print("🔴 TRANSLATION PIPELINE")
    print(f"{'-'*100}")
    
    print(f"\n🔄 Query translated: {row['translation_question_english']}")
    
    trans_hi = row['translation_response_hindi']
    print(f"\n✅ Hindi Response:")
    print(trans_hi)
    
    print(f"\n📋 Original English (before Hindi translation):")
    print(row['translation_response_english'])
    
    print(f"\n🔄 Back-translating Hindi to English...")
    trans_back = translate_text(trans_hi, src='hi', dest='en')
    print(f"\n📖 Back-translated:")
    print(trans_back)
    
    quality_trans = assess_response_quality(trans_hi)
    print(f"\n📈 Quality: {quality_trans['score']:.0%} ({quality_trans['word_count']} words)")
    print(f"   ✓ Length adequate: {'✅' if quality_trans['length_ok'] else '❌'}")
    print(f"   ✓ Not mock: {'✅' if quality_trans['not_mock'] else '❌'}")
    print(f"   ✓ Has medical terms: {'✅' if quality_trans['has_medical_terms'] else '❌'}")
    print(f"   ✓ Coherent: {'✅' if quality_trans['coherent'] else '❌'}")
    
    # COMPARISON (without df_evaluation - just show times and quality)
    print(f"\n{'-'*100}")
    print("📊 COMPARISON")
    print(f"{'-'*100}")
    print(f"Quality Scores: Multi={quality_multi['score']:.0%}, Trans={quality_trans['score']:.0%}")
    print(f"Word Count: Multi={quality_multi['word_count']}, Trans={quality_trans['word_count']}")
    print(f"Time: Multi={row['multilingual_total_time']:.2f}s, Trans={row['translation_total_time']:.2f}s")
    
    time.sleep(0.2)

# Summary
print("\n" + "=" * 100)
print("📊 OVERALL QUALITY SUMMARY")
print("=" * 100)

quality_scores_multi = []
quality_scores_trans = []

for idx, row in df_results.iterrows():
    quality_scores_multi.append(assess_response_quality(row['multilingual_response_hindi'])['score'])
    quality_scores_trans.append(assess_response_quality(row['translation_response_hindi'])['score'])

print(f"\nAverage Quality Scores:")
print(f"  Multilingual: {np.mean(quality_scores_multi):.1%}")
print(f"  Translation:  {np.mean(quality_scores_trans):.1%}")

print(f"\nHigh Quality Responses (≥75%):")
print(f"  Multilingual: {sum(s >= 0.75 for s in quality_scores_multi)}/{len(quality_scores_multi)}")
print(f"  Translation:  {sum(s >= 0.75 for s in quality_scores_trans)}/{len(quality_scores_trans)}")

print("\n" + "=" * 100)
print("✅ VERIFICATION COMPLETE!")
print("=" * 100)
print("\n💡 Note: Run Notebook 4 to get semantic similarity and ROUGE scores")


RESPONSE VERIFICATION WITH BACK-TRANSLATION

QUESTION 1/12

📝 Hindi: मधुमेह के प्रमुख लक्षण क्या हैं?
📝 English: What are the main symptoms of diabetes?

----------------------------------------------------------------------------------------------------
🔵 MULTILINGUAL EMBEDDINGS
----------------------------------------------------------------------------------------------------

✅ Hindi Response:
सामग्री में मधुमेह के प्रमुख रहस्यों के बारे में कोई जानकारी नहीं है।

🔄 Back-translating to English...

📖 Back-translated:
The content does not contain any information about the major secrets of diabetes.

📈 Quality: 75% (13 words)
   ✓ Length adequate: ❌
   ✓ Not mock: ✅
   ✓ Has medical terms: ✅
   ✓ Coherent: ✅

----------------------------------------------------------------------------------------------------
🔴 TRANSLATION PIPELINE
----------------------------------------------------------------------------------------------------

🔄 Query translated: What are the main symptoms of diab